In [1]:
import numpy as np
import random
import pandas as pd
import os

from common import OUTPUTPATH
from models import linear_pcc
import data

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# estimators here
linear_pcc.fit(data.x1, data.y)

In [3]:
num_importance = 100 # defult values show all variables
from sklearn.inspection import permutation_importance

def individual_importance_dataframe(est, X, y, scoring='neg_log_loss', num_importance=100000, n_repeats=100, seeds=None):
    """
    This algorithm use linear estimators and get the importance variables where p < 0.05
    To easy comparison, we set importance score: 1- p
    """
    r = permutation_importance(est, X, y, n_repeats=n_repeats, scoring=scoring, random_state=seeds)
    non_zero_indx = np.where(r.importances_mean > 0)
    indx = r.importances_mean[non_zero_indx].argsort()[::-1]
    important_variables = X.columns[non_zero_indx][indx]
    importance_scores = r.importances_mean[non_zero_indx][indx]
    coef = est.coef_[0][non_zero_indx][indx]
    df = pd.DataFrame({'variables': important_variables[:num_importance], 'coef': coef[:num_importance], 'importance': importance_scores[:num_importance]})
    return df

def linear_importance_dataframe(est, X, y, scoring = 'neg_log_loss', num_importance=100000, n_repeats=100, seeds=None):
    """
    Combine all morphologies plots together
    """
    data1 = pd.concat([X, y], axis=1)
    cnt = None
    for i in range(3):
        col_indx = -4 + i

        estimator, X, y = est[i], data1.iloc[:, :col_indx], data.y.iloc[:, i]
        temp_df = individual_importance_dataframe(est=estimator, X=X, y=y, scoring=scoring, num_importance=num_importance, n_repeats=n_repeats, seeds=seeds)
        if not cnt: 
            df = temp_df
            cnt = 1
        else:
            df = pd.concat([df, temp_df], axis=1)

    upper_columns = ['Sphere', 'Worm', 'Vesicle']
    lower_columns = ['variables', 'coef', 'importance']
    df.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
    return df

show all importance dataframe for Sphere, Worm, Vesicle

In [4]:
linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=100000, seeds=1000)

/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sk

Phase         Sphere                                Worm            \
Property   variables      coef    importance   variables      coef   
0         mw_tot_cre  0.000781  8.209574e+00  mw_tot_cre -0.000436   
1         mv_tot_cre -0.050703  4.059840e+00  mv_tot_cre  0.025036   
2             dp_cre -0.056522  3.846369e+00      dp_cre  0.027473   
3           apol_cna  0.257623  1.206764e+00  mw_tot_cna -0.000258   
4            psa_cna -0.037091  2.481660e-01    apol_cna -0.062277   
5             dp_cna  0.049359  2.358354e-01  mv_tot_cna  0.024094   
6         mv_tot_cna  0.040055  1.930769e-01        conc  0.072983   
7            psa_cre -0.206010  1.518237e-01      mw_cna  0.005097   
8               conc -0.127560  7.618014e-02     psa_cre  0.055504   
9         mw_tot_cna -0.000103  2.134203e-02     psa_cna  0.007698   
10            mw_cre  0.028013  1.673286e-02      mw_cre -0.024466   
11            mw_cna -0.003501  1.309211e-02        temp  0.019550   
12          apol_cre  0.019163  9.403013e-04      dp_cna  0.004539   
13         clogp_cna  0.020923  9.267362e-04      sphere -0.016023   
14              temp -0.011317  8.832596e-04   clogp_cna -0.008845   
15           charged  0.021135  2.894088e-04    apol_cre  0.002626   
16                ph -0.020380  1.628935e-04     charged -0.005067   
17         clogp_cre  0.011543  5.010610e-05   clogp_cre -0.004384   
18        mass_ratio  0.008822  4.832134e-05  mass_ratio -0.001474   
19         vol_ratio  0.008712  4.749440e-05   vol_ratio -0.001363   
20            mv_cre  0.000570  9.425541e-07      mv_cna  0.000923   
21               NaN       NaN           NaN        salt -0.000361   
22               NaN       NaN           NaN      mv_cre  0.000133   
23               NaN       NaN           NaN         NaN       NaN   
24               NaN       NaN           NaN         NaN       NaN   

Phase                      Vesicle                          
Property    importance   variables      coef    importance  
0         6.328502e+00  mw_tot_cre -0.000936  7.671937e+00  
1         1.611345e+00  mv_tot_cre  0.080782  7.356431e+00  
2         1.473559e+00      dp_cre  0.041079  3.334120e+00  
3         1.571045e-01  mw_tot_cna -0.000785  1.012136e+00  
4         1.377726e-01  mv_tot_cna  0.080923  5.775365e-01  
5         8.018805e-02    apol_cna -0.128724  3.906745e-01  
6         4.463982e-02     psa_cna  0.039046  3.099664e-01  
7         3.479748e-02      sphere -0.697121  5.224654e-02  
8         2.568971e-02   clogp_cna -0.194776  3.287992e-02  
9         1.864555e-02     psa_cre  0.086631  2.652419e-02  
10        7.281475e-03        conc  0.056344  2.321922e-02  
11        7.164378e-03      mw_cre -0.036125  1.494196e-02  
12        3.167173e-03          ph -0.142967  1.150382e-02  
13        8.056879e-04        temp  0.029290  1.137290e-02  
14        3.105275e-04        worm -0.248615  9.007197e-03  
15        1.172001e-04    apol_cre  0.042498  2.900999e-03  
16        9.176831e-05     charged -0.141321  1.524377e-03  
17        4.872024e-05  mass_ratio -0.104957  1.087386e-03  
18        6.538231e-06   vol_ratio -0.101589  1.022252e-03  
19        5.636020e-06      dp_cna -0.002743  6.512715e-04  
20        3.158118e-06      mw_cna -0.000616  2.192065e-04  
21        3.783603e-07      mv_cna  0.015011  7.137528e-05  
22        2.773271e-07   clogp_cre -0.023424  1.135012e-05  
23                 NaN        salt -0.004210  2.218469e-06  
24                 NaN      mv_cre  0.000180  2.659400e-07

In [5]:
top5_linear_dataframe = linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=5, seeds=1000)
top5_linear_dataframe.to_csv(os.path.join(OUTPUTPATH, 'Top5_linear_dataframe.csv'))
top5_linear_dataframe

/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sk

Phase         Sphere                             Worm                       \
Property   variables      coef importance   variables      coef importance   
0         mw_tot_cre  0.000781   8.209574  mw_tot_cre -0.000436   6.328502   
1         mv_tot_cre -0.050703   4.059840  mv_tot_cre  0.025036   1.611345   
2             dp_cre -0.056522   3.846369      dp_cre  0.027473   1.473559   
3           apol_cna  0.257623   1.206764  mw_tot_cna -0.000258   0.157105   
4            psa_cna -0.037091   0.248166    apol_cna -0.062277   0.137773   

Phase        Vesicle                       
Property   variables      coef importance  
0         mw_tot_cre -0.000936   7.671937  
1         mv_tot_cre  0.080782   7.356431  
2             dp_cre  0.041079   3.334120  
3         mw_tot_cna -0.000785   1.012136  
4         mv_tot_cna  0.080923   0.577537